In [81]:
import pandas as pd

In [82]:
df_raw = pd.read_csv('statements-data-semi-stacked.csv')

In [83]:
df = df_raw.copy()
for col in ['FY18', 'FY17', 'FY16', 'FY15']:
    df[col] = df[col].str.replace(',', '')
    df[col] = df[col].str.replace('-', '')
    df[col] = pd.to_numeric(df[col])

In [84]:
stacked = df.set_index(['Company', 'Metric']).stack().reset_index().sort_values('Company')
stacked.columns = ['Company', 'Metric', 'FY', 'Value']

In [87]:
stacked.tail()

,Company,Metric,FY,Value
221,TELUS,Equity,FY17,9416000.0
220,TELUS,Equity,FY18,10259000.0
39,TELUS,EBITDA,FY15,4050000.0
327,TELUS,FFO,FY17,3947000.0
219,TELUS,Long Term Debt,FY15,11182000.0


In [88]:
stacked.to_csv('stacked-statements-data.csv', index=False)

In [89]:
pivoted = stacked.pivot_table(columns='Metric', index=['Company', 'FY'], values = 'Value')

In [91]:
pivoted['EBIT to Interest Expense'] = pivoted['EBIT'] / pivoted['Interest Expense']
pivoted['EBITDA to Interest Expense'] = pivoted['EBITDA'] / pivoted['Interest Expense']
pivoted['EBIT Margin'] = pivoted['EBIT'] / pivoted['Total Revenue']
pivoted['EBITDA Margin'] = pivoted['EBITDA'] / pivoted['Total Revenue']
pivoted['Debt to EBITDA'] = pivoted['Total Debt'] / pivoted['EBITDA']
pivoted['Debt to EBIT'] = pivoted['Total Debt'] / pivoted['EBIT']
pivoted['Debt to Equity'] = pivoted['Total Debt'] / pivoted['Equity']
pivoted['ROE'] = pivoted['Net Income'] / pivoted['Equity']
pivoted['FFO + IntExp to IntExp'] =  (pivoted['FFO'] +  pivoted['Interest Expense']) /  pivoted['Interest Expense']
pivoted['FFO to Debt'] =  pivoted['FFO'] /  pivoted['Total Debt']


In [92]:
averages = pivoted.groupby('Company')[pivoted.columns].mean()
averages.index = zip(averages.index, ['Average']*len(averages.index))

In [93]:
pivoted = pd.concat([pivoted, averages]).sort_index()

In [95]:
pivoted.tail()

Metric               Assets       EBIT     EBITDA      Equity        FFO  \
Company FY                                                                 
TELUS   Average  29563250.0  2705500.0  4336000.0   8816000.0  3695000.0   
        FY15     26406000.0  2537000.0  4050000.0   7672000.0  3556000.0   
        FY16     27729000.0  2617000.0  4228000.0   7917000.0  3219000.0   
        FY17     31053000.0  2781000.0  4450000.0   9416000.0  3947000.0   
        FY18     33065000.0  2887000.0  4616000.0  10259000.0  4058000.0   

Metric           Interest Expense  Long Term Debt  Net Income  Total Debt  \
Company FY                                                                  
TELUS   Average          544000.0      12089250.0   1441000.0  20711500.0   
        FY15             470000.0      11182000.0   1382000.0  18734000.0   
        FY16             502000.0      11625000.0   1223000.0  19793000.0   
        FY17             579000.0      12332000.0   1559000.0  21595000.0   
        FY18             625000.0      13218000.0   1600000.0  22724000.0   

Metric           Total Revenue  EBIT to Interest Expense  \
Company FY                                                 
TELUS   Average     13138750.0                  5.008332   
        FY15        12430000.0                  5.397872   
        FY16        12725000.0                  5.213147   
        FY17        13305000.0                  4.803109   
        FY18        14095000.0                  4.619200   

Metric           EBITDA to Interest Expense  EBIT Margin  EBITDA Margin  \
Company FY                                                                
TELUS   Average                    8.027649     0.205901       0.330009   
        FY15                       8.617021     0.204103       0.325825   
        FY16                       8.422311     0.205658       0.332259   
        FY17                       7.685665     0.209019       0.334461   
        FY18                       7.385600     0.204824       0.327492   

Metric           Debt to EBITDA  Debt to EBIT  Debt to Equity       ROE  \
Company FY                                                                
TELUS   Average        4.770694      7.645973        2.362599  0.164036   
        FY15           4.625679      7.384312        2.441867  0.180136   
        FY16           4.681410      7.563240        2.500063  0.154478   
        FY17           4.852809      7.765192        2.293437  0.165569   
        FY18           4.922877      7.871147        2.215031  0.155961   

Metric           FFO + IntExp to IntExp  FFO to Debt  
Company FY                                            
TELUS   Average                7.822008     0.178450  
        FY15                   8.565957     0.189815  
        FY16                   7.412351     0.162633  
        FY17                   7.816926     0.182774  
        FY18                   7.492800     0.178578

In [96]:
pivoted.to_csv('statements-data-pivoted.csv')